In [2]:
import pickle
import os
import numpy as np
import pandas as pd

from keras.layers import Dense, Dropout, Embedding
from keras.models import Input, Model
from keras.preprocessing import sequence
from tensorflow.python.framework import ops
from tcn import TCN
from sklearn.model_selection import train_test_split
from keras.models import load_model

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

ModuleNotFoundError: No module named 'keras'

In [35]:
def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

In [30]:
data = pickle.load(open( "dfsm.pkl", "rb" ))
cols=list(data[1].iloc[:,:34].columns)+list(data[1].iloc[:,-6:-1])

In [31]:
x=list()
y=list()
for i in range(len(data)):
    x.append(np.asarray(data[i][cols[:35]]))
    y.append(np.asarray(data[i].iloc[:,-1:]))
        
x_norm=list()
for i in range(len(x)):
    x_norm.append((x[i]+0.000000000000000000001)/(x[i].max(axis=0)+0.000000000000000000001))

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.3,random_state=30)

In [33]:
def get_x_y_shift(x_train,y_train):
    while True:
        for k in range(len(x_train)):
            x_t = x_train[k]
            y_t = y_train[k]
            sample_weights=5*(y_t)+1
            yield (np.expand_dims(np.array(x_t),axis=0), np.expand_dims(np.array(y_t),axis=0),sample_weights.T)

In [34]:
gen = get_x_y_shift(x_train=x_train,y_train=y_train)

In [36]:
i = Input(batch_shape=(1, None, 35))

o = TCN(nb_filters=30, kernel_size=3, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32,64], padding='causal', use_skip_connections=True, dropout_rate=0.0, return_sequences=True, name='tcn')(i)
o = Dense(1,activation='sigmoid')(o)

model = Model(inputs=[i], outputs=[o])
model.compile(optimizer='adam',loss='mean_squared_error', metrics=['binary_accuracy'],sample_weight_mode='temporal')

In [ ]:
model.fit_generator(gen, epochs=2, steps_per_epoch=len(x_train), max_queue_size=100, verbose=1)

Epoch 1/2
19936/28235 [====================>.........] - ETA: 2:19 - loss: 0.1611 - binary_accuracy: 0.9654

In [ ]:
compute_prediction_utility()